In [5]:
import pandas as pd
import numpy as np
import os
import requests

## Sprawdzanie po trans_formed

In [167]:
import os
os.getcwd()

'c:\\Users\\Tomek\\Desktop\\PW MINI IAD\\Semestr 6\\Hurtownie\\ChicagoCrashesETL'

In [170]:
fact_crash = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_crash.pkl')
dim_vehicles = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_vehicle.pkl')
dim_crash_info = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_crash_info.pkl')
fact_weather = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_weather.pkl')
dim_people = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_people.pkl')
dim_date = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_date.pkl')

In [ ]:
# crash_key_map = fact_crash.set_index("CRASH_RECORD_ID")["FACT_CRASH_KEY"]
dim_people_cp = dim_people.copy()

# dim_people_cp.insert(1, "CRASH_KEY", dim_people_cp["CRASH_RECORD_ID"].map(crash_key_map))

fact_crash_expanded = fact_crash.merge(dim_people_cp[["CRASH_RECORD_ID", "PERSON_ID", "VEHICLE_ID"]], on="CRASH_RECORD_ID", how='inner')
fact_crash_expanded = fact_crash_expanded.merge(dim_crash_info[["CRASH_RECORD_ID", "CRASH_INFO_KEY"]], on='CRASH_RECORD_ID', how='inner')
fact_crash_expanded

# jeszcze location_id

,FACT_CRASH_KEY,CRASH_RECORD_ID,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,date_id,PERSON_ID,VEHICLE_ID,CRASH_INFO_KEY
0,1153394593627935104,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,2,1,1,0,0,0,1,0,2019112609,O790106,750401,7017221841570418401
1,1153394593627935104,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,2,1,1,0,0,0,1,0,2019112609,O790107,-1,7017221841570418401
2,913253574156243668,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,2,1,0,0,1,0,1,0,2019092718,O747999,710658,13479415490444100987
3,913253574156243668,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,2,1,0,0,1,0,1,0,2019092718,O748000,-1,13479415490444100987
4,4125836884551549824,4502c89bb17d2a4817c517541350b1470636cc9c29e6fd...,2,0,0,0,0,0,2,0,2019021906,O587406,560000,17995927666335292266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263997,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,P180099,747774,18382781804148259871
263998,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,P180101,747781,18382781804148259871
263999,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,P182839,758707,18382781804148259871
264000,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,P182840,758708,18382781804148259871


In [184]:
dim_crash_info.head(5)

,CRASH_INFO_KEY,CRASH_RECORD_ID,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY
0,7017221841570418401,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,NO CONTROLS,NO CONTROLS,PEDESTRIAN,ONE-WAY,CURVE ON GRADE,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,NOT APPLICABLE,FATAL
1,13479415490444100987,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,NO CONTROLS,NO CONTROLS,PEDESTRIAN,FOUR WAY,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,NONINCAPACITATING INJURY
2,17995927666335292266,4502c89bb17d2a4817c517541350b1470636cc9c29e6fd...,NO CONTROLS,NO CONTROLS,REAR END,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"$501 - $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,NO INDICATION OF INJURY
3,9657579150507369252,46fa1015e8c466dde2a9a5db8a3b9f6e95e84f2f066386...,NO CONTROLS,NO CONTROLS,PARKED MOTOR VEHICLE,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,NOT APPLICABLE,NO INDICATION OF INJURY
4,203320938600502705,4e661d1348b236fc49d6f3299e61098d55a0b9506e8f54...,NO CONTROLS,NO CONTROLS,REAR END,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",NOT APPLICABLE,NOT APPLICABLE,NO INDICATION OF INJURY


In [139]:
dim_people.head(5)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT VALUE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816,09/28/2019 03:30:00 AM,M,25,NONE PRESENT,UNKNOWN,NONE,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,-999.0
1,O663866,DRIVER,f09cdb96b5c9d25a1f82573805eb40067ac1555bc35cda...,632057,06/06/2019 12:40:00 PM,F,-1,SAFETY BELT USED,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,-999.0
2,O848861,DRIVER,d08b8b938cc6777e1d3f5a5c299649829d812336fb6679...,805584,11/09/2019 03:20:00 PM,M,56,UNKNOWN,UNKNOWN,NONE,NO INDICATION OF INJURY,IMPROPER LANE CHANGE,UNKNOWN,UNKNOWN,TEST NOT OFFERED,-999.0
3,O850909,DRIVER,cd68e3218ffe768cfc1a5e0c75fd3a54365ef1d53e67c5...,807488,10/20/2019 05:20:00 PM,M,46,UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,-999.0
4,O851790,DRIVER,9a87e7f29da8636e57cbd77bcb69a3cc352ef4976b02bb...,808333,07/30/2019 10:30:00 AM,M,60,UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,IMPROPER BACKING,NOT OBSCURED,NORMAL,TEST NOT OFFERED,-999.0


In [141]:
dim_date

,date_id,full_date,year,quarter,month,month_name,day_of_month,day_of_week,day_name,is_weekend,week_of_year,is_holiday,holiday_name
0,20160101,2016-01-01,2016,1,1,January,1,5,Friday,0,53,1,New Year's Day
1,20160102,2016-01-02,2016,1,1,January,2,6,Saturday,1,53,0,None
2,20160103,2016-01-03,2016,1,1,January,3,7,Sunday,1,53,0,None
3,20160104,2016-01-04,2016,1,1,January,4,1,Monday,0,1,0,None
4,20160105,2016-01-05,2016,1,1,January,5,2,Tuesday,0,1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,20211227,2021-12-27,2021,4,12,December,27,1,Monday,0,52,0,None
2188,20211228,2021-12-28,2021,4,12,December,28,2,Tuesday,0,52,0,None
2189,20211229,2021-12-29,2021,4,12,December,29,3,Wednesday,0,52,0,None
2190,20211230,2021-12-30,2021,4,12,December,30,4,Thursday,0,52,0,None


0

In [9]:
fact_crash = pd.read_pickle("/Users/julia/Desktop/semestr-6/magazyny/ChicagoCrashesETL/airflow-postgres-docker/data/tmp/transformed/fact_crash.pkl")

In [12]:
fact_crash.head(20)

,FACT_CRASH_KEY,CRASH_RECORD_ID,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,date_id
0,102052558975241319,032ca7e09951de4385eec78a50a5f9f33b217838578b29...,3,0,0,0,0,0,1,0,2020010100
1,2989301628946909366,71065ad347ee197080534e112b27c8a5b6fb8d9bbde2c3...,1,2,0,0,2,0,1,0,2020010100
2,15759330137532455643,9c2724305c03bf461d72cbcf2879c96b86ed5e180b62be...,1,-1,-1,-1,-1,-1,-1,-1,2020010100
3,18411092908018610329,3187e5839fab7ff654383ba44a2e5b430c856b3fa10dab...,4,0,0,0,0,0,7,0,2020010100
4,18009739063765005411,449bc7511dba6e6d605411276dc08dbeeccff39b074db0...,2,0,0,0,0,0,1,0,2020010100
5,17528604748918565077,6792e30369a62483f115d4900c9225b17fdca2dc3547f6...,2,0,0,0,0,0,3,0,2020010100
6,4101458230553004886,2cf6f0a893d1cb9f1a0bba9435cd9e1938e2b0a278ffb3...,2,0,0,0,0,0,2,0,2020010100
7,7758129516194055061,9c2ddefdd8c9c409db174e219b0511db3a4c06490e1bcc...,2,0,0,0,0,0,3,0,2020010100
8,14024785583040875057,d100ea227bccd86263d7cbdc1550b24ed1955cffa085ca...,1,0,0,0,0,0,1,0,2020010100
9,16431909946148874362,b73c19ba7e38f0e966410bea19ea5d2d2b75f72b8f2529...,3,0,0,0,0,0,1,0,2020010100
